In [1]:
import requests
import re
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from bs4 import BeautifulSoup

In [2]:
dataframe = pd.read_excel('/Users/kim/Desktop/180708_search_terms.xlsx', sheetname=0)
mylist = dataframe['Terms'].tolist()
print (mylist)

['VSV', 'oncolytic virus', 'SMAC', 'IAP', 'Kras', 'VEGF', 'BET', 'Bromodomain', 'Erbb', 'PD-1', 'CTLA-4', 'Vista', 'Lag-3', 'Checkpoint']


In [3]:
df = pd.DataFrame()
# create empty data frame
rows = mylist
columns = mylist
condition = 'Cancer'
rows = [r.replace(' ', '+') for r in rows]
columns = [c.replace(' ', '+') for c in columns]
for i in rows:
    newrow = []
    for j in columns:
        if i == j:
            link = 'https://clinicaltrials.gov/ct2/results?recrs=&cond=' + condition + '&term=' + i + '&cntry=&state=&city=&dist='
            #print (link)
            r1 = requests.get(link)
            soup = BeautifulSoup(r1.text, 'html.parser')
            hits_text = soup.find(style="margin-top:1ex;margin-left:8px;margin-right:8px;").text
            try:
                hits_number = int(re.findall(r'([0-9]+)', hits_text)[0])
                newrow.append(hits_number)
            except IndexError:
                hits_number = 0         
                newrow.append(hits_number)
        else:
            link = 'https://clinicaltrials.gov/ct2/results?recrs=&cond=' + condition + '&term=' + i + '+' + j + '&cntry=&state=&city=&dist='
            #print (link)
            r1 = requests.get(link)
            soup = BeautifulSoup(r1.text, 'html.parser')
            hits_text = soup.find(style="margin-top:1ex;margin-left:8px;margin-right:8px;").text
            try:
                hits_number = int(re.findall(r'^([0-9]+)', hits_text)[0])
                newrow.append(hits_number)
            except IndexError:
                hits_number = 0
                newrow.append(hits_number)
    df = df.append(pd.Series(newrow), ignore_index=True)
# problem with no study found
df.index = rows
df.columns = columns
df

,VSV,oncolytic+virus,SMAC,IAP,Kras,VEGF,BET,Bromodomain,Erbb,PD-1,CTLA-4,Vista,Lag-3,Checkpoint
VSV,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oncolytic+virus,2.0,79.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,9.0,4.0,0.0,0.0,6.0
SMAC,0.0,0.0,10.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
IAP,0.0,0.0,1.0,14.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
Kras,0.0,3.0,0.0,0.0,496.0,25.0,1.0,1.0,15.0,3.0,2.0,1.0,0.0,3.0
VEGF,0.0,0.0,1.0,1.0,25.0,1290.0,0.0,0.0,10.0,25.0,4.0,1.0,0.0,3.0
BET,0.0,0.0,0.0,0.0,1.0,0.0,35.0,18.0,0.0,4.0,1.0,0.0,0.0,1.0
Bromodomain,0.0,0.0,0.0,0.0,1.0,0.0,18.0,28.0,0.0,1.0,0.0,0.0,0.0,0.0
Erbb,0.0,0.0,0.0,1.0,15.0,10.0,0.0,0.0,236.0,1.0,0.0,2.0,0.0,0.0
PD-1,0.0,9.0,1.0,1.0,3.0,25.0,4.0,1.0,1.0,1006.0,90.0,2.0,23.0,143.0


In [4]:
writer = pd.ExcelWriter('LKS180919.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()